In [1]:
# Import DeepInsight
import sys
sys.path.insert(0, "/home/marx/Documents/Github/DeepInsight")
import deepinsight
# Choose GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import h5py
%load_ext autoreload
%autoreload 2

In [2]:
fp_deepinsight = './test_files/test.h5'
input_length = int(3e5)
input_channels = 5
sampling_rate = 30000
input_output_ratio = 100

np.random.seed(0)
rand_input = np.sin(np.random.rand(int(input_length), input_channels))
rand_input_timesteps = np.arange(0, input_length)
rand_output = np.random.rand(input_length // input_output_ratio)
rand_timesteps = np.arange(0, input_length, input_output_ratio)

print(rand_input[0,0])
print(rand_input_timesteps[0:10])
print(rand_output[0])
print(rand_timesteps[0:10])

0.5216753442582844
[0 1 2 3 4 5 6 7 8 9]
0.7039485667507264
[  0 100 200 300 400 500 600 700 800 900]


In [3]:
# Transform raw data to frequency domain
deepinsight.preprocess.preprocess_input(fp_deepinsight, rand_input, sampling_rate=sampling_rate)


/home/marx/miniconda3/envs/deepinsight/lib/python3.7/site-packages/wavelets/transform.py:104: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  wavelet_data[slices],


Number of chunks 5
Starting chunk 0
Start 0 - End 100000
This chunk time 4.527534484863281
Starting chunk 1
Start 50000 - End 150000
This chunk time 2.225909948348999
Starting chunk 2
Start 100000 - End 200000
This chunk time 2.2600257396698
Starting chunk 3
Start 150000 - End 250000
This chunk time 2.211815595626831
Starting chunk 4
Start 200000 - End 300000
This chunk time 2.171374559402466


In [6]:
hdf5_file = h5py.File(fp_deepinsight, mode='r')

# Get size of wavelets
input_wavelets = hdf5_file['inputs/wavelets']
# Check statistics of wavelets
np.testing.assert_almost_equal(np.mean(input_wavelets), 0.048329726)
np.testing.assert_almost_equal(np.std(input_wavelets), 0.032383125)
np.testing.assert_almost_equal(np.median(input_wavelets), 0.04608967)
np.testing.assert_almost_equal(np.max(input_wavelets), 0.40853173)
np.testing.assert_almost_equal(np.min(input_wavelets), 1.6544704e-05)
hdf5_file.close()

In [7]:
# Prepare outputs
deepinsight.preprocess.preprocess_output(fp_deepinsight, rand_input_timesteps, rand_output,
                                           rand_timesteps)

(300, 1)


In [8]:
# Define loss functions and train model
loss_functions = {'output_aligned' : 'mse'}
loss_weights = {'output_aligned' : 1}
user_opts = {'epochs' : 2, 'steps_per_epoch' : 10, 'validation_steps' : 10, 'log_output' : False, 'save_model' : False}

deepinsight.train.run_from_path(fp_deepinsight, loss_functions, loss_weights, user_opts)

------------------------------------------------
-> Running test.h5 from ./test_files
- Logs : ./test_files/logs/test 
- Model temporary : ./test_files/models/tmp/tmp_model 
- Model : ./test_files/models/test_model.h5
------------------------------------------------
------------------------------------------------
Starting standard model
------------------------------------------------


/home/marx/Documents/Github/DeepInsight/deepinsight/util/hdf5.py:115: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(path) as f:


InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
# Get loss and shuffled loss for influence plot, both is also stored back to HDF5 file
losses, output_predictions, indices = deepinsight.analyse.get_model_loss(fp_deepinsight, stepsize=10)

In [ ]:
shuffled_losses = deepinsight.analyse.get_shuffled_model_loss(fp_deepinsight, axis=1,stepsize=10)

In [ ]:
deepinsight.visualize.plot_residuals(fp_deepinsight, frequency_spacing=2,
                                     output_names=['output_aligned'])

In [ ]:
# from tqdm.notebook import tqdm
# import time
# for i1 in tqdm(range(5), ncols='50%'):
#     for i2 in tqdm(range(30), leave=False, ncols='25%'):
#         # do something, e.g. sleep
#         time.sleep(0.1)